In [55]:
#import necessary libraries
!pip install requests
import requests

In [74]:
import pandas as pd
import requests
from datetime import datetime
asteroids_data = []                                      #create an empty list
target = 10000
API_KEY = "XlHLTgH0OoY9dT6codFgzxF8ULuioQUDbTBf9WsQ"     #URL obtained from NASA official website
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"

while len(asteroids_data) < target:      
    response = requests.get(url)
    data = response.json() 
    details = data['near_earth_objects']

    for date, ast_details in details.items():
        for ast in ast_details:                
                 asteroids_data.append(dict(             #append values from the URL to the dictionary
                            id = int(ast['id']),
                            name = ast['name'],
                            absolute_magnitude_h = float(ast['absolute_magnitude_h']),
                            estimated_diameter_min_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                            estimated_diameter_max_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                            is_potentially_hazardous_asteroid  = ast['is_potentially_hazardous_asteroid'],
                            neo_reference_id = int(ast['neo_reference_id']),
                            close_approach_date = ast['close_approach_data'][0]['close_approach_date'],
                            relative_velocity_kmph = float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                            astronomical = float(ast['close_approach_data'][0]['miss_distance']['astronomical']),
                            miss_distance_km = float(ast['close_approach_data'][0]['miss_distance']['kilometers']),
                            miss_distance_lunar = float(ast['close_approach_data'][0]['miss_distance']['lunar']),
                            orbiting_body = ast['close_approach_data'][0]['orbiting_body']))



                 if len(asteroids_data) == target:
                        break
        if len(asteroids_data) == target:
                        break
    url = data['links'].get('next')                     #pagination 

df = pd.DataFrame(asteroids_data)                       #create a dataframe

In [ ]:
df['close_approach_date'].unique()

In [82]:
#install sql 
pip install pymysql                                     

Note: you may need to restart the kernel to use updated packages.


In [84]:
import pandas as pd 
import numpy as np
import pymysql 

In [86]:
#create connection to the sql
conn=pymysql.connect(host="127.0.0.1",user="root",passwd="Arahim4199") 

In [88]:
#create a cursor
curs = conn.cursor()                                                   

In [90]:
#create a database named NASA_NEO_PROJECT
curs.execute("create database NASA_NEO_PROJECT")                       

1

In [92]:
# create a connection to the database
conn = pymysql.connect(host = '127.0.0.1', user='root',passwd='Arahim4199',database='NASA_NEO_PROJECT') 

In [94]:
curs =conn.cursor()

In [96]:
#create a table named ASTEROID
curs.execute("create table ASTEROID (ID INT,NAME VARCHAR(30),ABSOLUTE_MAGNITUDE float,EST_DIA_MIN_KM float,EST_DIA_MAX_KM float, IS_HAZARDOUS  boolean)")                    

0

In [102]:
#insert values from the dataframe to the created table
asteroid_insert ='''insert into ASTEROID(ID,NAME,ABSOLUTE_MAGNITUDE,EST_DIA_MIN_KM,EST_DIA_MAX_KM,IS_HAZARDOUS) values(%s,%s,%s,%s,%s,%s)'''
for i in asteroids_data:
    values=(i['id'],i['name'],i['absolute_magnitude_h'],i['estimated_diameter_min_km'],i['estimated_diameter_max_km'],i['is_potentially_hazardous_asteroid'])
    curs.execute(asteroid_insert,values)

In [104]:
#check for proper execution
pd.read_sql_query('select * from asteroid limit 5',conn) 

/var/folders/5j/dqld7g010pn_g3494p0p7yqw0000gn/T/ipykernel_1245/4293095395.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('select * from asteroid limit 5',conn)


,ID,NAME,ABSOLUTE_MAGNITUDE,EST_DIA_MIN_KM,EST_DIA_MAX_KM,IS_HAZARDOUS
0,2415949,415949 (2001 XY10),19.37,0.355267,0.794401,0
1,3160747,(2003 SR84),26.00,0.016771,0.037501,0
2,3309828,(2005 YQ96),20.62,0.199781,0.446725,1
3,3457842,(2009 HC21),22.10,0.101054,0.225964,0
4,3553062,(2010 XA11),26.10,0.016016,0.035813,0


In [106]:
#to confirm changes
conn.commit()

In [108]:
#create a table named CLOSE_APPROACH_DATA
curs.execute("create table CLOSE_APPROACH_DATA(NEO_REF_ID INT,CLOSE_APPROACH_DATE DATE,RELATIVE_VELOCITY float,ASTRONOMICAL float, MISS_DISTANCE_KM float,MISS_DISTANCE_LUNAR float,ORBITING_BODY VARCHAR(30))")

0

In [112]:
#insert values from the dataframe to the created table
cda_insert ='''insert into CLOSE_APPROACH_DATA(NEO_REF_ID,CLOSE_APPROACH_DATE,RELATIVE_VELOCITY,ASTRONOMICAL, MISS_DISTANCE_KM,MISS_DISTANCE_LUNAR,ORBITING_BODY) values(%s,%s,%s,%s,%s,%s,%s)'''
for i in asteroids_data:
    values=(i['neo_reference_id'],i['close_approach_date'],i['relative_velocity_kmph'],i['astronomical'],i['miss_distance_km'],i['miss_distance_lunar'],i['orbiting_body'])
    curs.execute(cda_insert,values)

In [114]:
#check for proper execution
pd.read_sql_query('select * from close_approach_data limit 5',conn)

/var/folders/5j/dqld7g010pn_g3494p0p7yqw0000gn/T/ipykernel_1245/3732776865.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('select * from close_approach_data limit 5',conn)


,NEO_REF_ID,CLOSE_APPROACH_DATE,RELATIVE_VELOCITY,ASTRONOMICAL,MISS_DISTANCE_KM,MISS_DISTANCE_LUNAR,ORBITING_BODY
0,2415949,2024-01-02,57205.9,0.337254,50452400.0,131.1920,Earth
1,3160747,2024-01-02,38589.1,0.132343,19798200.0,51.4813,Earth
2,3309828,2024-01-02,56413.0,0.167013,24984700.0,64.9679,Earth
3,3457842,2024-01-02,21891.1,0.492051,73609800.0,191.4080,Earth
4,3553062,2024-01-02,31469.0,0.235802,35275500.0,91.7271,Earth


In [118]:
#to confirm changes 
conn.commit()

In [2]:
#to exit sql connection
conn.close()

NameError: name 'conn' is not defined